In [1]:
import re
import nltk
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
def pre_processing(sentence):
    return remove_stopwords(tokenize_sentence(remove_punctuation(sentence)))

# Remove punctuation from a list of words
def remove_punctuation(sentence):
    return re.sub(r'[^\w\s]', '', sentence)

# Remove stopwords from a list of words
def remove_stopwords(words_list):
    stopwords = open("stop_words_FULL.txt", "r")
    stopwords_list = []
    for word in stopwords:
        stopwords_list.append(word.replace('\n', ''))
    stopwords.close()
    return [value.lower() for value in words_list if value.lower() not in stopwords_list]

# Tokenize the input sentence and also lemmatize its words
def tokenize_sentence(sentence):
    words_list = []
    lmtzr = WordNetLemmatizer()
    for tag in nltk.pos_tag(word_tokenize(sentence)):
        if (tag[1][:2] == "NN"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.NOUN))
        elif (tag[1][:2] == "VB"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.VERB))
        elif (tag[1][:2] == "RB"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.ADV))
        elif (tag[1][:2] == "JJ"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.ADJ))
    return words_list

# Union of the pre-processed words of the definitions and terms from the examples in WN for a sense.
def get_signature(sense):
    signature = []
    for word in tokenize_sentence(sense.definition()):  # definition tokenization
        signature.append(word)
    for example in sense.examples():  # example tokenization
        for word in tokenize_sentence(example):
            # Merge definition and examples
            signature.append(word)
    return signature

Read corpus file. It return a list of list of documents and words for each document.

In [3]:
def read_corpus(txt_file):
    
    with open(txt_file, encoding='utf-8') as file:
        # for each doc create list of pre-processed words in that doc (list of lists)
        documents_words = []

        for line in file:
            if "<doc" in line:  # tag for new doc
                document_words = []  # list of words that will be part of the document
                while True:
                    next_line = file.readline()  # read next line

                    # remove unuseful tags
                    next_line_proc = next_line.replace("<p> ", "").replace("</p>\n", "").replace("/p", "")

                    if "</doc>" in next_line:
                        break
                    # pre-processing steps
                    sentence_words = pre_processing(next_line_proc)
                    document_words.extend(sentence_words)
                documents_words.append(document_words)
        file.close()

    print("Documents number: ", len(documents_words))
    
    return documents_words

Topic modelling using gensim

In [4]:
from gensim import corpora, models

topic_num = 10
topic_words_num = 5

def topic_modelling(documents_words):
    
    # Create a dict with integer keys for all words
    dictionary_LDA = corpora.Dictionary(documents_words)

    # delete all terms that do NOT appear in at least 3 documents.
    # delete all terms that appear in more than 60% of documents (see filter_extremes official doc).
    dictionary_LDA.filter_extremes(no_below=3, no_above=0.6)

    # Converts each document into a list of BoW (list of (id_term, term_frequency) for each term in doc)
    corpus_idbow_freq = [dictionary_LDA.doc2bow(document_words) for document_words in documents_words]
    
    # https://radimrehurek.com/gensim/models/ldamodel.html
    lda_model = models.LdaModel(corpus_idbow_freq, num_topics=topic_num, \
                                id2word=dictionary_LDA, \
                                passes=3, alpha=[0.01] * topic_num, \
                                eta=[0.01] * len(dictionary_LDA.keys()))
    
    return lda_model, corpus_idbow_freq

Show topics.

In [5]:
documents_words = read_corpus("travelling.txt")

model, corpus_idbow_freq = topic_modelling(documents_words)

topics = {'Topic ' + str(i): [(token, round(score, 3)) for token, score in model.show_topic(i, topn=topic_words_num)] for i in range(0, model.num_topics)}
for key, value in topics.items():
    print(key,":", topics[key])

Documents number:  100
Topic 0 : [('book', 0.024), ('hotel', 0.022), ('beach', 0.019), ('holiday', 0.015), ('travel', 0.011)]
Topic 1 : [('travel', 0.022), ('hotel', 0.02), ('place', 0.015), ('money', 0.014), ('stay', 0.012)]
Topic 2 : [('learn', 0.02), ('word', 0.019), ('language', 0.019), ('student', 0.015), ('article', 0.012)]
Topic 3 : [('__________________', 0.027), ('students', 0.026), ('function', 0.015), ('exam', 0.011), ('activity', 0.01)]
Topic 4 : [('clause', 0.029), ('happen', 0.013), ('situation', 0.011), ('conditionals', 0.011), ('dont', 0.01)]
Topic 5 : [('lesson', 0.014), ('speak', 0.013), ('clause', 0.013), ('students', 0.012), ('audio', 0.01)]
Topic 6 : [('clause', 0.022), ('word', 0.019), ('noun', 0.013), ('tense', 0.011), ('dailystep', 0.011)]
Topic 7 : [('students', 0.063), ('exam', 0.033), ('book', 0.028), ('sb', 0.025), ('speak', 0.025)]
Topic 8 : [('clause', 0.033), ('result', 0.027), ('example', 0.027), ('conditionals', 0.02), ('third', 0.02)]
Topic 9 : [('less

Show topics for each document.

In [6]:
print ("Documents topic list")
for i in range (0, len(corpus_idbow_freq)):
    print ("Doc", i, ":", model[corpus_idbow_freq[i]])

Documents topic list
Doc 0 : [(0, 0.2397877), (2, 0.53992265), (6, 0.10147474), (8, 0.1178163)]
Doc 1 : [(0, 0.15899965), (1, 0.19565934), (2, 0.1550823), (6, 0.062708415), (7, 0.42739)]
Doc 2 : [(6, 0.99791145)]
Doc 3 : [(2, 0.61039203), (6, 0.08974457), (7, 0.111245535), (9, 0.18827704)]
Doc 4 : [(5, 0.7216771), (6, 0.27816293)]
Doc 5 : [(2, 0.99993885)]
Doc 6 : [(8, 0.92606527), (9, 0.07305638)]
Doc 7 : [(2, 0.42251316), (4, 0.22539023), (9, 0.3515822)]
Doc 8 : [(1, 0.4828723), (2, 0.517053)]
Doc 9 : [(1, 0.44506282), (2, 0.55466694)]
Doc 10 : [(8, 0.9998743)]
Doc 11 : [(4, 0.95521915), (8, 0.044536207)]
Doc 12 : [(5, 0.99913543)]
Doc 13 : [(9, 0.9996692)]
Doc 14 : [(6, 0.6021036), (7, 0.21609908), (9, 0.18141067)]
Doc 15 : [(2, 0.23615946), (3, 0.7275523), (8, 0.01815119), (9, 0.018039074)]
Doc 16 : [(8, 0.99928623)]
Doc 17 : [(1, 0.21587253), (2, 0.09368496), (6, 0.69024813)]
Doc 18 : [(4, 0.7258934), (6, 0.27380815)]
Doc 19 : [(4, 0.91771317), (8, 0.08197072)]
Doc 20 : [(1, 0.364